# **LIBRARIES IMPORT**

In [25]:
from bs4 import BeautifulSoup
import requests
import json
import os
import re
from dotenv import load_dotenv
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
from urllib.parse import quote_plus
from nltk.tokenize import word_tokenize, sent_tokenize, RegexpTokenizer, TreebankWordTokenizer, wordpunct_tokenize
import nltk
from nltk.corpus import stopwords
from nltk.stem.isri import ISRIStemmer
from nltk.stem import WordNetLemmatizer
import clean
import stem
import utils
import tagger
import pyarabic.araby as araby
from tashaphyne.stemming import ArabicLightStemmer
import qalsadi.lemmatizer
import http.client
from farasa.pos import FarasaPOSTagger 
from farasa.ner import FarasaNamedEntityRecognizer 
from farasa.diacratizer import FarasaDiacritizer 
from farasa.segmenter import FarasaSegmenter 
from farasa.stemmer import FarasaStemmer

# **DATA**

## **Scraping data**

In [3]:
BASE_URL = 'https://www.aljazeera.net'

articles = []
articlesTitles = []
articlesLinks = []
articlesDates = []
articlesContents = []

html = requests.get(BASE_URL + '/news/').text
soup = BeautifulSoup(html, "html.parser")

newsFeedContainer = soup.find('section', {'id': 'news-feed-container'})
print('News feed container found!')
for a in newsFeedContainer.find_all('a', href=True):
    articlesLinks.append(a['href'])
    articlesTitles.append(a.text)

articlesLinks = [(i, l) for i, l in enumerate(articlesLinks) if l.startswith('/news/')]
articlesTitles = [t for i, t in enumerate(articlesTitles) if i in [i for i, l in articlesLinks]]
articlesLinks = [l for i, l in articlesLinks]

print('Articles links retrieved!')
print('Articles titles retrieved!')
for b in articlesLinks:
    url = BASE_URL + b
    print('Retrieving article content from: ' + url)
    html = requests.get(url).text
    soup = BeautifulSoup(html, "html.parser")
    article = ''

    articleDate = soup.find('span', {'class': 'article-dates__published'}).text

    articlesParagraphsContainer = soup.find('div', {'class': 'wysiwyg wysiwyg--all-content css-1vkfgk0'})

    articlesParagraphs = articlesParagraphsContainer.find_all('p', recursive=False)

    articlesDates.append(articleDate)

    for p in articlesParagraphs:
        article += p.text + '\n'

    articlesContents.append(article)
print('Articles dates retrieved!')
print('Articles Contents retrieved!')
for i in range(len(articlesTitles)):
    articles.append({
        'title': articlesTitles[i],
        'link': articlesLinks[i],
        'date': articlesDates[i],
        'content': articlesContents[i]
    })

News feed container found!
Articles links retrieved!
Articles titles retrieved!
Retrieving article content from: https://www.aljazeera.net/news/2024/4/4/%d8%a7%d8%b4%d8%aa%d8%a8%d8%a7%d9%83%d8%a7%d8%aa-%d8%b9%d9%86%d9%8a%d9%81%d8%a9-%d8%a8%d8%ae%d8%a7%d9%86-%d9%8a%d9%88%d9%86%d8%b3-%d9%88%d8%a7%d9%84%d8%a7%d8%ad%d8%aa%d9%84%d8%a7%d9%84
Retrieving article content from: https://www.aljazeera.net/news/2024/4/4/%d8%b7%d8%a8%d9%8a%d8%a8-%d8%a5%d8%b3%d8%b1%d8%a7%d8%a6%d9%8a%d9%84%d9%8a-%d8%a3%d9%82%d8%af%d8%a7%d9%85-%d8%a7%d9%84%d9%85%d8%b9%d8%aa%d9%82%d9%84%d9%8a%d9%86-%d9%85%d9%86-%d8%ba%d8%b2%d8%a9
Retrieving article content from: https://www.aljazeera.net/news/2024/4/4/%d8%a7%d9%84%d8%a3%d8%b2%d9%87%d8%b1-%d8%a3%d8%b5%d9%88%d8%a7%d8%aa-%d9%88%d8%b5%d9%81%d8%aa-%d8%a7%d9%84%d8%a5%d8%b3%d9%84%d8%a7%d9%85-%d8%a8%d8%a7%d9%84%d8%a5%d8%b1%d9%87%d8%a7%d8%a8
Retrieving article content from: https://www.aljazeera.net/news/2024/4/4/%d8%a3%d9%85-%d8%aa%d8%aa%d9%87%d8%ac%d9%85-%d8%b9%d9%84%d9%89-%d9

## **Saving data in a local file**
If you are on Google Colab, you will need to make the `tmp` folder first. 


In [3]:
# If you are on Google Colab:
# with open('/content/tmp/aljazzera.json', 'w', encoding='utf-8') as json_file:
with open('tmp/aljazzera.json', 'w', encoding='utf-8') as json_file:
    json.dump(articles, json_file, indent=4, ensure_ascii=False)

## **Saving data in mongodb**
To save the data in mongodb, you will need to use your own credentials.

```shell
cp .env.example .env
```

Then you will need to fill the `.env` file with your credentials.

In [4]:
load_dotenv()

username = quote_plus(os.getenv('MONGODB_USERNAME'))
password = quote_plus(os.getenv('MONGODB_PASS'))
cluster = os.getenv('MONGODB_CLUSTER_ID')

uri = "mongodb+srv://" + username + ":" + password + "@cluster." + cluster + ".mongodb.net/?retryWrites=true&w=majority"

client = MongoClient(uri, server_api=ServerApi('1'))

try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


In [5]:
db = client['nlp-lab-db']

collection = db['aljazeera-articles']

collection.insert_many(articles)

print('Articles inserted successfully')

Articles inserted successfully


Saving the data in a mongodb is a good practice in the real world. It is a good way to store the data and make it available for future use. But for this lab it is just a demonstration of how to save data in a mongodb. We will be just using data from the local file.

In [2]:
# If you are on Google Colab:
# with open('/content/tmp/aljazzera.json', 'r', encoding='utf-8') as json_file:
with open('tmp/aljazzera.json', 'r', encoding='utf-8') as json_file:
    articles = json.load(json_file)
    articleExample = articles[0]

articleExample

{'title': 'ترامب يصف المهاجرين بأنهم "حيوانات"',
 'link': '/news/2024/4/3/%d8%aa%d8%b1%d8%a7%d9%85%d8%a8-%d9%8a%d8%b5%d9%81-%d8%a7%d9%84%d9%85%d9%87%d8%a7%d8%ac%d8%b1%d9%8a%d9%86-%d8%a8%d8%a3%d9%86%d9%87%d9%85-%d8%ad%d9%8a%d9%88%d8%a7%d9%86%d8%a7%d8%aa',
 'date': '3/4/2024',
 'content': 'وصف المرشح الجمهوري للرئاسة الأميركية، دونالد ترامب، المهاجرين غير الشرعيين في الولايات المتحدة بأنهم "حيوانات" و"ليسوا بشرا" خلال خطاب ألقاه في ولاية ميشيغان، مما أثار انتقادات واسعة لعودته إلى اللغة المهينة التي اعتمدها في حملاته السابقة.\nوجاءت تصريحات ترامب خلال ظهوره إلى جانب عدد من ضباط إنفاذ القانون، حيث تطرق إلى عدة قضايا جنائية تتعلق بمشتبه بهم موجودين في الولايات المتحدة بشكل غير قانوني، وحذر من أن العنف والفوضى سيدمران البلاد إذا لم يحقق الفوز في الانتخابات المقررة في نوفمبر/تشرين الثاني.\nوفي خطاب آخر في ولاية ويسكونسن، واصل ترامب نبرة التصريحات المثيرة، حيث وصف انتخابات عام 2024 بأنها "المعركة النهائية" للأمة.\nوفيما يتعلق بحادثة لاكين رايلي، طالبة التمريض البالغة من العمر 22 عاما من جورجي

In [3]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\legion\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\legion\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\legion\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# **DATA WRANGLING & PROCESSING PIPELINE**
This is the most important part of the project. Here we will be cleaning the data and making it ready for the analysis.
Our pipeline will have the following steps:
1. **Removing Punctuation**
2. **Discretization**
3. **Tokenization**
4. **Normalization**
5. **Removing Stopwords**

The pipeline input will be an article in string format and the output will be a list of words.

Each step of the pipeline will print its output for better understanding.

We will also be adding necessary processing if needed.

### **Removing punctuation, discretization, normalization and paragraphs tokenization**

In [4]:
content = articleExample['content']

content = clean.remove_punctuations(content)
content = clean.remove_diacritics(content)
content = clean.normalize_arabic(content)

contentParagraphs = content.split('\n')

contentParagraphs = list(filter(None, contentParagraphs))
res = []
for ele in contentParagraphs:
    if ele.strip():
        res.append(ele)

contentParagraphs = res
contentParagraphs

['وصف المرشح الجمهوري للرءاسه الاميركيه دونالد ترامب المهاجرين غير الشرعيين في الولايات المتحده بانهم حيوانات وليسوا بشرا خلال خطاب القاه في ولايه ميشيغان مما اثار انتقادات واسعه لعودته الي اللغه المهينه التي اعتمدها في حملاته السابقه',
 'وجاءت تصريحات ترامب خلال ظهوره الي جانب عدد من ضباط انفاذ القانون حيث تطرق الي عده قضايا جناءيه تتعلق بمشتبه بهم موجودين في الولايات المتحده بشكل غير قانوني وحذر من ان العنف والفوضي سيدمران البلاد اذا لم يحقق الفوز في الانتخابات المقرره في نوفمبرتشرين الثاني',
 'وفي خطاب اخر في ولايه ويسكونسن واصل ترامب نبره التصريحات المثيره حيث وصف انتخابات عام 2024 بانها المعركه النهاءيه للامه',
 'وفيما يتعلق بحادثه لاكين رايلي طالبه التمريض البالغه من العمر 22 عاما من جورجيا والتي يزعم انها قتلت علي يد مهاجر غير شرعي من فنزويلا اشار ترامب الي ان بعض المهاجرين ليسوا بشرا',
 'وفي تعليقه قال ترامب يقول الديمقراطيون من فضلكم لا تطلقوا عليهم اسم الحيوانات انهم بشر انا اقول لا انهم ليسوا بشرا انهم حيوانات',
 'وفي خطاباته المثيره للجدل يدعي ترامب في كثير من الاحيان ان ال

### **Sentences tokenization**

In [5]:
for p in contentParagraphs:
    p = sent_tokenize(p)
    print(p)

['وصف المرشح الجمهوري للرءاسه الاميركيه دونالد ترامب المهاجرين غير الشرعيين في الولايات المتحده بانهم حيوانات وليسوا بشرا خلال خطاب القاه في ولايه ميشيغان مما اثار انتقادات واسعه لعودته الي اللغه المهينه التي اعتمدها في حملاته السابقه']
['وجاءت تصريحات ترامب خلال ظهوره الي جانب عدد من ضباط انفاذ القانون حيث تطرق الي عده قضايا جناءيه تتعلق بمشتبه بهم موجودين في الولايات المتحده بشكل غير قانوني وحذر من ان العنف والفوضي سيدمران البلاد اذا لم يحقق الفوز في الانتخابات المقرره في نوفمبرتشرين الثاني']
['وفي خطاب اخر في ولايه ويسكونسن واصل ترامب نبره التصريحات المثيره حيث وصف انتخابات عام 2024 بانها المعركه النهاءيه للامه']
['وفيما يتعلق بحادثه لاكين رايلي طالبه التمريض البالغه من العمر 22 عاما من جورجيا والتي يزعم انها قتلت علي يد مهاجر غير شرعي من فنزويلا اشار ترامب الي ان بعض المهاجرين ليسوا بشرا']
['وفي تعليقه قال ترامب يقول الديمقراطيون من فضلكم لا تطلقوا عليهم اسم الحيوانات انهم بشر انا اقول لا انهم ليسوا بشرا انهم حيوانات']
['وفي خطاباته المثيره للجدل يدعي ترامب في كثير من الاحيان ان ال

### **Removing stopwords and words tokenization**

In [6]:
stop_words = set(stopwords.words('arabic'))
contentWords = []

for p in contentParagraphs:
    words = word_tokenize(p)
    print("original:")
    print(words)
    words = [w for w in words if not w in stop_words]
    print("without stopwords:")
    print(words)
    print("--------------------------------------")
    contentWords.append(words)

original:
['وصف', 'المرشح', 'الجمهوري', 'للرءاسه', 'الاميركيه', 'دونالد', 'ترامب', 'المهاجرين', 'غير', 'الشرعيين', 'في', 'الولايات', 'المتحده', 'بانهم', 'حيوانات', 'وليسوا', 'بشرا', 'خلال', 'خطاب', 'القاه', 'في', 'ولايه', 'ميشيغان', 'مما', 'اثار', 'انتقادات', 'واسعه', 'لعودته', 'الي', 'اللغه', 'المهينه', 'التي', 'اعتمدها', 'في', 'حملاته', 'السابقه']
without stopwords:
['وصف', 'المرشح', 'الجمهوري', 'للرءاسه', 'الاميركيه', 'دونالد', 'ترامب', 'المهاجرين', 'الشرعيين', 'الولايات', 'المتحده', 'بانهم', 'حيوانات', 'وليسوا', 'بشرا', 'خلال', 'خطاب', 'القاه', 'ولايه', 'ميشيغان', 'اثار', 'انتقادات', 'واسعه', 'لعودته', 'الي', 'اللغه', 'المهينه', 'اعتمدها', 'حملاته', 'السابقه']
--------------------------------------
original:
['وجاءت', 'تصريحات', 'ترامب', 'خلال', 'ظهوره', 'الي', 'جانب', 'عدد', 'من', 'ضباط', 'انفاذ', 'القانون', 'حيث', 'تطرق', 'الي', 'عده', 'قضايا', 'جناءيه', 'تتعلق', 'بمشتبه', 'بهم', 'موجودين', 'في', 'الولايات', 'المتحده', 'بشكل', 'غير', 'قانوني', 'وحذر', 'من', 'ان', 'العنف', 'والفوض

# **Stemming and Lemmatization**
## **Stemming - implementation**

In [7]:
contentWordsStemmed = []

for pw in contentWords:
    words = stem.light_stem(pw)
    print(words)
    contentWordsStemmed.append(words)

['وصف', 'مرشح', 'جمهوري', 'رءاسه', 'اميركيه', 'دونالد', 'ترامب', 'مهاجر', 'شرعي', 'ولاي', 'متحده', 'بان', 'حيوان', 'وليس', 'بشرا', 'خلال', 'خطاب', 'قاه', 'ولايه', 'ميشيغ', 'اثار', 'انتقاد', 'واسعه', 'لعودته', 'الي', 'لغه', 'مهينه', 'اعتمد', 'حملاته', 'سابقه']
['وجاءت', 'تصريح', 'ترامب', 'خلال', 'ظهوره', 'الي', 'جانب', 'عدد', 'ضباط', 'انفاذ', 'قان', 'تطرق', 'الي', 'عده', 'قضا', 'جناءيه', 'تتعلق', 'بمشتبه', 'موجود', 'ولاي', 'متحده', 'بشكل', 'قانو', 'وحذر', 'ان', 'عنف', 'فوضي', 'سيدمر', 'بلاد', 'اذا', 'يحقق', 'فوز', 'انتخاب', 'مقرره', 'نوفمبرتشر', 'ثاني']
['وفي', 'خطاب', 'اخر', 'ولايه', 'ويسكونسن', 'واصل', 'ترامب', 'نبره', 'تصريح', 'مثيره', 'وصف', 'انتخاب', 'عام', '2024', 'بان', 'معركه', 'نهاءيه', 'امه']
['وفي', 'يتعلق', 'بحادثه', 'لاك', 'رايلي', 'طالبه', 'تمريض', 'بالغه', 'عمر', '22', 'عاما', 'جورج', 'والتي', 'يزعم', 'انها', 'قتلت', 'علي', 'يد', 'مهاجر', 'شرعي', 'فنزويلا', 'اشار', 'ترامب', 'الي', 'ان', 'مهاجر', 'بشرا']
['وفي', 'تعليقه', 'قال', 'ترامب', 'يقول', 'ديمقراطي', 'فضل', 'تطلق', 

## **Lemmatization**

In [8]:
lemmatizer = WordNetLemmatizer()
contentWordsLemmatized = []

for pw in contentWords:
    words = [lemmatizer.lemmatize(w) for w in pw]
    print(words)
    contentWordsLemmatized.append(words)

['وصف', 'المرشح', 'الجمهوري', 'للرءاسه', 'الاميركيه', 'دونالد', 'ترامب', 'المهاجرين', 'الشرعيين', 'الولايات', 'المتحده', 'بانهم', 'حيوانات', 'وليسوا', 'بشرا', 'خلال', 'خطاب', 'القاه', 'ولايه', 'ميشيغان', 'اثار', 'انتقادات', 'واسعه', 'لعودته', 'الي', 'اللغه', 'المهينه', 'اعتمدها', 'حملاته', 'السابقه']
['وجاءت', 'تصريحات', 'ترامب', 'خلال', 'ظهوره', 'الي', 'جانب', 'عدد', 'ضباط', 'انفاذ', 'القانون', 'تطرق', 'الي', 'عده', 'قضايا', 'جناءيه', 'تتعلق', 'بمشتبه', 'موجودين', 'الولايات', 'المتحده', 'بشكل', 'قانوني', 'وحذر', 'ان', 'العنف', 'والفوضي', 'سيدمران', 'البلاد', 'اذا', 'يحقق', 'الفوز', 'الانتخابات', 'المقرره', 'نوفمبرتشرين', 'الثاني']
['وفي', 'خطاب', 'اخر', 'ولايه', 'ويسكونسن', 'واصل', 'ترامب', 'نبره', 'التصريحات', 'المثيره', 'وصف', 'انتخابات', 'عام', '2024', 'بانها', 'المعركه', 'النهاءيه', 'للامه']
['وفيما', 'يتعلق', 'بحادثه', 'لاكين', 'رايلي', 'طالبه', 'التمريض', 'البالغه', 'العمر', '22', 'عاما', 'جورجيا', 'والتي', 'يزعم', 'انها', 'قتلت', 'علي', 'يد', 'مهاجر', 'شرعي', 'فنزويلا', 'اشار',

## **Stemming with araby**

In [39]:
ArListem = ArabicLightStemmer()

contentWordsStemmed = []

for pw in contentWords:
    words = [ArListem.light_stem(w) for w in pw]
    print(words)
    contentWordsStemmed.append(words)

['صف', 'مرشح', 'جمهور', 'رءاسه', 'اميركيه', 'دونالد', 'ترامب', 'مهاجر', 'شرعي', 'ولا', 'متحده', 'ان', 'حيوان', 'ليسوا', 'شرا', 'خلال', 'خطاب', 'قاه', 'اي', 'ميشيغ', 'ثار', 'انتقاد', 'سع', 'عود', 'لي', 'لغه', 'مهينه', 'عتمد', 'حمل', 'سابقه']
['جاء', 'تصريح', 'ترامب', 'خلال', 'ظهور', 'لي', 'جانب', 'عدد', 'ضباط', 'نفاذ', 'قان', 'طرق', 'لي', 'عد', 'قضاي', 'جناء', 'تعلق', 'مشتب', 'موجود', 'ولا', 'متحده', 'شكل', 'قان', 'حذر', 'ان', 'عنف', 'فوض', 'دمر', 'بلاد', 'ذا', 'حقق', 'فوز', 'انتخاب', 'مقرره', 'نوفمبرتشرين', 'ثان']
['في', 'خطاب', 'خر', 'اي', 'يسكونسن', 'صل', 'ترامب', 'بر', 'تصريح', 'مثيره', 'صف', 'انتخاب', 'عام', '2024', 'ان', 'معركه', 'نهاءيه', 'امه']
['فيما', 'تعلق', 'حادث', 'اك', 'رايل', 'طالب', 'تمريض', 'بالغه', 'عمر', '22', 'عام', 'جورجي', 'التي', 'زعم', 'نه', 'قتل', 'على', 'يد', 'مهاجر', 'شرع', 'نزويل', 'شار', 'ترامب', 'لي', 'ان', 'مهاجر', 'شرا']
['في', 'عليق', 'قال', 'ترامب', 'قول', 'الديمقراطيون', 'ضل', 'طلق', 'على', 'سم', 'حيوان', 'نهم', 'شر', 'نا', 'قول', 'نهم', 'شرا', 'نهم', 

## **Lemmatization with qalsadi**

In [9]:
lemmer = qalsadi.lemmatizer.Lemmatizer()

contentWordsLemmatized = []

for pw in contentWords:
    words = [lemmer.lemmatize(w) for w in pw]
    print(words)
    contentWordsLemmatized.append(words)

['صف', 'مرشح', 'جمهور', 'للرءاسه', 'الاميركيه', 'دونالد', 'ترامب', 'مهاجر', 'شرعي', 'ولاية', 'المتحده', 'بان', 'حيوان', 'ليسوا', 'بشر', 'خلال', 'خطاب', 'القاه', 'ولايه', 'ميشيغان', 'اثار', 'انتقاد', 'واسع', 'عود', 'ال', 'اللغه', 'المهينه', 'اعتمد', 'حمل', 'السابقه']
['جاء', 'تصريح', 'ترامب', 'خلال', 'ظهور', 'ال', 'جانب', 'عدد', 'ضباط', 'انفاذ', 'قانون', 'تطرق', 'ال', 'عد', 'قضايا', 'جناءيه', 'تعلق', 'مشتبه', 'موجود', 'ولاية', 'المتحده', 'شكل', 'قانون', 'حذر', 'ان', 'عنف', 'والفوضي', 'دمر', 'بلاد', 'اذا', 'أحق', 'فوز', 'انتخاب', 'المقرره', 'نوفمبرتشرين', 'ثاني']
['في', 'خطاب', 'اخر', 'ولايه', 'ويسكونسن', 'واصل', 'ترامب', 'نبر', 'تصريح', 'المثيره', 'صف', 'انتخاب', 'عام', '2024', 'بان', 'المعركه', 'النهاءيه', 'لام']
['فيما', 'تعلق', 'حادث', 'لاك', 'رايلي', 'طالب', 'تمريض', 'البالغه', 'عمر', '22', 'عام', 'جورجي', 'التي', 'زعم', 'انها', 'قتل', 'علي', 'يد', 'مهاجر', 'شرع', 'فنزويلا', 'اشار', 'ترامب', 'ال', 'ان', 'مهاجر', 'بشر']
['في', 'تعليق', 'قال', 'ترامب', 'أقول', 'ديمقراطي', 'فضل', 'طلق'

## **Stemming with farasa**
Farasa is a fast and accurate modern Arabic NLP toolkit, developed by [Arabic Language Technologies Group](https://alt.qcri.org/) at [Qatar Computing Research Institute (QCRI)](https://www.hbku.edu.qa/en/qcri). The python package we are using is a wrapper around the original Java package since the original package is not developed in python. 

In [10]:
farasaStemmer = FarasaStemmer()

#### **Stemming tokens with farasa**

In [ ]:
# NOT RECOMMENDED DUE TO THE TIME IT TAKES AND PERFORMANCE
contentWordsStemmed = []

for pw in contentWords:
    words = [farasaStemmer.stem(w) for w in pw]
    print(words)
    contentWordsStemmed.append(words)

The Farasa library is so HEAVY computationally, so using it on each token is not a good idea. Instead, we can use it on the whole text. This way, we can reduce the number of calls to the library. Then we will apply the same preprocessing steps as we done first.

That's why we will stem a whole text, POS tag it, and then apply the same preprocessing steps as we did before.

In [12]:
stemmedTextFarasa = farasaStemmer.stem(content)
print("Stemmed Text:\n", stemmedTextFarasa)

Stemmed Text:
 وصف مرشح جمهوري رءاسه اميركيه دونالد ترامب مهاجر غير شرعي في ولاية المتحده أن حيوان ليس بشر خلال خطاب ألقى في لا ميشيغان من اثار انتقاد واسع عودة الي لغه مهينه الذي اعتمد في حملة سابق
جاء تصريح ترامب خلال ظهور الي جانب عدد من ضابط إنفاذ قانون حيث تطرق الي عد قضية جناءيه تعلق مشتبه ب موجود في ولاية المتحده شكل غير قانوني حذر من ان عنف فوضي دمر بلد اذ لم حقق فوز في انتخاب مقرره في نوفمبرتشرين ثاني
في خطاب آخر في لا ويسكونسن واصل ترامب نبر تصريح مثيره حيث وصف انتخاب عام 2024 أن معركه نهاءيه لم
في تعلق حادث لاك رايلي طالب تمريض بالغ من عمر 22 عام من جورجيا الذي زعم ان قتل علي يد مهاجر غير شرعي من فنزويلا أشار ترامب الي ان بعض مهاجر ليس بشر
في تعليق قال ترامب قال ديموقراطي من فضل لا أطلق على اسم حيوان ان بشر أنا أقول لا ان ليس بشر ان حيوان


في خطاب مثيره جدل ادعى ترامب في كثير من حين ان مهاجر الذي عبر حد مع مكسيك شكل غير قانوني قد فر من سجن ملاجء في بلد اصليه من زاد من جراءم عنف في ولاية المتحده
اتهم منافس في انتخاب رءاسيه رءيس جو بايدن تحفيز جمهوري في كونجرس علي عدم إقرار ت

# **POS TAGGING & NER**
## **POS tagging with Farasa**
#### **POS tagging the uncleaned text**

In [13]:
pos_tagger = FarasaPOSTagger()
POSTaggedText = pos_tagger.tag(stemmedTextFarasa)

print("sample POS Tagged\n", POSTaggedText)

sample POS Tagged
 S/S وصف/V مرشح/NOUN-MS جمهوري/ADJ-MS رءاس/NOUN-MS +ه/PRON اميركي/NOUN-MS +ه/PRON دونالد/NOUN-MS ترامب/NOUN-MS مهاجر/NOUN-MS غير/NOUN-MS شرعي/ADJ-MS في/PREP ولاي +ة/NOUN+NSUFF-FS المتحده/NOUN-MS أن/PART حيوان/NOUN-MS ليس/V بشر/NOUN-MS خلال/NOUN-MS خطاب/NOUN-MS ألقى/V في/PREP لا/PART ميشيغان/NOUN-MS من/PREP اثار/NOUN-FS انتقاد/NOUN-MS واسع/ADJ-MS عود +ة/NOUN+NSUFF-FS الي/FOREIGN لغ/NOUN-MS +ه/PRON مهين/ADJ-MS +ه/PRON الذي/PART اعتمد/V في/PREP حمل +ة/NOUN+NSUFF-FS سابق/ADJ-MS E/E 
S/S جاء/V تصريح/NOUN-MS ترامب/NOUN-MS خلال/NOUN-MS ظهور/NOUN-MS الي/FOREIGN جانب/NOUN-MS عدد/NOUN-MS من/PREP ضابط/NOUN-MS إنفاذ/NOUN-MS قانون/NOUN-MS حيث/PART تطرق/V الي/FOREIGN عد/NOUN-MS قضي +ة/NOUN+NSUFF-FS جناءيه/NOUN-MS تعلق/V مشتبه/NOUN-MS ب/PREP موجود/ADJ-MS في/PREP ولاي +ة/NOUN+NSUFF-FS المتحده/NOUN-MS شكل/NOUN-MS غير/NOUN-MS قانوني/ADJ-MS حذر/ADJ-MS من/PREP ان/NOUN-MS عنف/NOUN-MS فوضي/ADJ-MS دمر/V بلد/NOUN-MS إذ/PART لم/PART حقق/V فوز/NOUN-MS في/PREP انتخاب/NOUN-MS مقرر/ADJ-MS +ه/PRON

In [14]:
POSTaggedParagraphs = POSTaggedText.split('\n')

POSTaggedParagraphs = list(filter(None, POSTaggedParagraphs))
res = []
for ele in POSTaggedParagraphs:
    if ele.strip():
        res.append(ele)

POSTaggedParagraphs = res
POSTaggedParagraphs

['S/S وصف/V مرشح/NOUN-MS جمهوري/ADJ-MS رءاس/NOUN-MS +ه/PRON اميركي/NOUN-MS +ه/PRON دونالد/NOUN-MS ترامب/NOUN-MS مهاجر/NOUN-MS غير/NOUN-MS شرعي/ADJ-MS في/PREP ولاي +ة/NOUN+NSUFF-FS المتحده/NOUN-MS أن/PART حيوان/NOUN-MS ليس/V بشر/NOUN-MS خلال/NOUN-MS خطاب/NOUN-MS ألقى/V في/PREP لا/PART ميشيغان/NOUN-MS من/PREP اثار/NOUN-FS انتقاد/NOUN-MS واسع/ADJ-MS عود +ة/NOUN+NSUFF-FS الي/FOREIGN لغ/NOUN-MS +ه/PRON مهين/ADJ-MS +ه/PRON الذي/PART اعتمد/V في/PREP حمل +ة/NOUN+NSUFF-FS سابق/ADJ-MS E/E ',
 'S/S جاء/V تصريح/NOUN-MS ترامب/NOUN-MS خلال/NOUN-MS ظهور/NOUN-MS الي/FOREIGN جانب/NOUN-MS عدد/NOUN-MS من/PREP ضابط/NOUN-MS إنفاذ/NOUN-MS قانون/NOUN-MS حيث/PART تطرق/V الي/FOREIGN عد/NOUN-MS قضي +ة/NOUN+NSUFF-FS جناءيه/NOUN-MS تعلق/V مشتبه/NOUN-MS ب/PREP موجود/ADJ-MS في/PREP ولاي +ة/NOUN+NSUFF-FS المتحده/NOUN-MS شكل/NOUN-MS غير/NOUN-MS قانوني/ADJ-MS حذر/ADJ-MS من/PREP ان/NOUN-MS عنف/NOUN-MS فوضي/ADJ-MS دمر/V بلد/NOUN-MS إذ/PART لم/PART حقق/V فوز/NOUN-MS في/PREP انتخاب/NOUN-MS مقرر/ADJ-MS +ه/PRON في/PREP نوفم

In [15]:
POSDict = utils.tagsDict(POSTaggedParagraphs, 'pos')

for w in POSDict:
    print(w)

{'S': 'S', 'وصف': 'V', 'مرشح': 'NOUN-MS', 'جمهوري': 'ADJ-MS', 'رءاس': 'NOUN-MS', '+ه': 'PRON', 'اميركي': 'NOUN-MS', 'دونالد': 'NOUN-MS', 'ترامب': 'NOUN-MS', 'مهاجر': 'NOUN-MS', 'غير': 'NOUN-MS', 'شرعي': 'ADJ-MS', 'في': 'PREP', 'ولاي+ة': 'NOUN+NSUFF-FS', 'المتحده': 'NOUN-MS', 'أن': 'PART', 'حيوان': 'NOUN-MS', 'ليس': 'V', 'بشر': 'NOUN-MS', 'خلال': 'NOUN-MS', 'خطاب': 'NOUN-MS', 'ألقى': 'V', 'لا': 'PART', 'ميشيغان': 'NOUN-MS', 'من': 'PREP', 'اثار': 'NOUN-FS', 'انتقاد': 'NOUN-MS', 'واسع': 'ADJ-MS', 'عود+ة': 'NOUN+NSUFF-FS', 'الي': 'FOREIGN', 'لغ': 'NOUN-MS', 'مهين': 'ADJ-MS', 'الذي': 'PART', 'اعتمد': 'V', 'حمل+ة': 'NOUN+NSUFF-FS', 'سابق': 'ADJ-MS', 'E': 'E'}
{'S': 'S', 'جاء': 'V', 'تصريح': 'NOUN-MS', 'ترامب': 'NOUN-MS', 'خلال': 'NOUN-MS', 'ظهور': 'NOUN-MS', 'الي': 'FOREIGN', 'جانب': 'NOUN-MS', 'عدد': 'NOUN-MS', 'من': 'PREP', 'ضابط': 'NOUN-MS', 'إنفاذ': 'NOUN-MS', 'قانون': 'NOUN-MS', 'حيث': 'PART', 'تطرق': 'V', 'عد': 'NOUN-MS', 'قضي+ة': 'NOUN+NSUFF-FS', 'جناءيه': 'NOUN-MS', 'تعلق': 'V', 'مشت

#### **POS tagging the stemmed text**

In [21]:
POSTaggedParagraphs = []

StemmedContentParagraphs = content.split('\n')
StemmedContentParagraphs = list(filter(None, StemmedContentParagraphs))

res = []
for ele in StemmedContentParagraphs:
    if ele.strip():
        res.append(ele)

StemmedContentParagraphs = res

for p in StemmedContentParagraphs:
        p = sent_tokenize(p)

for p in StemmedContentParagraphs:
    POSTaggedParagraphs.append(pos_tagger.tag(p))
    
POSTaggedParagraphs

['S/S وصف/V ال+ مرشح/DET+NOUN-MS ال+ جمهوري/DET+ADJ-MS ل+/PREP ال+ رءاسه/DET+NOUN-MS ال+ اميركيه/DET+ADJ-MS دونالد/NOUN-MS ترامب/NOUN-MS ال+ مهاجر +ين/DET+NOUN+NSUFF-MP غير/PART ال+ شرعي +ين/DET+ADJ+NSUFF-FD في/PREP ال+ ولاي +ات/DET+NOUN+NSUFF-FP المتحده/NOUN-MS ب+/PREP أن/PART +هم/PRON حيوان +ات/NOUN+NSUFF-FP و+/CONJ ليس +وا/V+PRON بشر/NOUN-MS +ا/CASE خلال/NOUN-MS خطاب/NOUN-MS ألقا/V +ه/PRON في/PREP و+/CONJ لا/PART +ي/PRON +ه/PRON ميشيغان/NOUN-MS مما/PREP+PART اثار/V انتقاد +ات/NOUN+NSUFF-MP واسع/ADJ-MS +ه/PRON ل+/PREP عود +ت/NOUN+NSUFF-FD +ه/PRON الي/FOREIGN ال+ لغه/DET+NOUN-MS ال+ مهينه/DET+ADJ-MS التي/PART اعتمد/V +ها/PRON في/PREP حمل +ات/NOUN+NSUFF-FP +ه/PRON ال+ سابقه/DET+NOUN-MS E/E',
 'S/S و+/CONJ جاء +ت/V+PRON تصريح +ات/NOUN+NSUFF-FP ترامب/NOUN-MS خلال/NOUN-MS ظهور/NOUN-MS +ه/PRON الي/FOREIGN جانب/NOUN-MS عدد/NOUN-MS من/PREP ضباط/NOUN-MP إنفاذ/NOUN-MS ال+ قانون/DET+NOUN-MS حيث/PART تطرق/V الي/FOREIGN عد/V +ه/PRON قضايا/NOUN-FP جناءيه/NOUN-MS تتعلق/V ب+/PREP مشتبه/NOUN-MS ب+/PR

In [22]:
POSDict = utils.tagsDict(POSTaggedParagraphs, 'pos')

for w in POSDict:
    print(w)

{'S': 'S', 'وصف': 'V', 'ال+مرشح': 'DET+NOUN-MS', 'ال+جمهوري': 'DET+ADJ-MS', 'ل+': 'PREP', 'ال+رءاسه': 'DET+NOUN-MS', 'ال+اميركيه': 'DET+ADJ-MS', 'دونالد': 'NOUN-MS', 'ترامب': 'NOUN-MS', 'ال+مهاجر+ين': 'DET+NOUN+NSUFF-MP', 'غير': 'PART', 'ال+شرعي+ين': 'DET+ADJ+NSUFF-FD', 'في': 'PREP', 'ال+ولاي+ات': 'DET+NOUN+NSUFF-FP', 'المتحده': 'NOUN-MS', 'ب+': 'PREP', 'أن': 'PART', '+هم': 'PRON', 'حيوان+ات': 'NOUN+NSUFF-FP', 'و+': 'CONJ', 'ليس+وا': 'V+PRON', 'بشر': 'NOUN-MS', '+ا': 'CASE', 'خلال': 'NOUN-MS', 'خطاب': 'NOUN-MS', 'ألقا': 'V', '+ه': 'PRON', 'لا': 'PART', '+ي': 'PRON', 'ميشيغان': 'NOUN-MS', 'مما': 'PREP+PART', 'اثار': 'V', 'انتقاد+ات': 'NOUN+NSUFF-MP', 'واسع': 'ADJ-MS', 'عود+ت': 'NOUN+NSUFF-FD', 'الي': 'FOREIGN', 'ال+لغه': 'DET+NOUN-MS', 'ال+مهينه': 'DET+ADJ-MS', 'التي': 'PART', 'اعتمد': 'V', '+ها': 'PRON', 'حمل+ات': 'NOUN+NSUFF-FP', 'ال+سابقه': 'DET+NOUN-MS', 'E': 'E'}
{'S': 'S', 'و+': 'CONJ', 'جاء+ت': 'V+PRON', 'تصريح+ات': 'NOUN+NSUFF-FP', 'ترامب': 'NOUN-MS', 'خلال': 'NOUN-MS', 'ظهور': 

#### **POS tagging the lemmatized text**

In [23]:
POSTaggedParagraphs = []

TEXT = [' '.join(p) for p in contentWordsLemmatized]

for p in TEXT:
    POSTaggedParagraphs.append(pos_tagger.tag(p))
    
POSTaggedParagraphs

['S/S صف/NOUN-MS مرشح/ADJ-MS جمهور/NOUN-MS ل+/PREP ال+ رءاسه/DET+NOUN-MS ال+ اميركيه/DET+ADJ-MS دونالد/NOUN-MS ترامب/NOUN-MS مهاجر/NOUN-MS شرعي/ADJ-MS ولاي +ة/NOUN+NSUFF-FS المتحده/NOUN-MS بان/NOUN-MP حيوان/NOUN-MS ليس +وا/V+PRON بشر/NOUN-MS خلال/NOUN-MS خطاب/NOUN-MS ألقا/V +ه/PRON و+/CONJ لا/PART +ي/PRON +ه/PRON ميشيغان/NOUN-MS اثار/NOUN-FS انتقاد/NOUN-MS واسع/ADJ-MS عود/NOUN-MS ال/DET ال+ لغه/DET+ADJ-MS ال+ مهينه/DET+ADJ-MS اعتمد/V حمل/NOUN-MS ال+ سابقه/DET+NOUN-MS E/E',
 'S/S جاء/V تصريح/NOUN-MS ترامب/NOUN-MS خلال/NOUN-MS ظهور/NOUN-MS ال/DET جانب/NOUN-MS عدد/NOUN-MS ضباط/NOUN-MP إنفاذ/NOUN-MS قانون/NOUN-MS تطرق/V ال/DET عد/NOUN-MS قضايا/NOUN-FP جناءيه/NOUN-MS تعلق/V مشتبه/NOUN-MS موجود/ADJ-MS ولاي +ة/NOUN+NSUFF-FS المتحده/NOUN-MS شكل/NOUN-MS قانون/NOUN-MS حذر/V ان/NOUN-MS عنف/NOUN-MS و+/CONJ ال+ فوضي/DET+NOUN-MS دمر/V بلاد/NOUN-FP اذ/NOUN-MS +ا/CASE أحق/ADJ-MS فوز/NOUN-MS انتخاب/NOUN-MS ال+ مقرره/DET+NOUN-MS نوفمبرتشرين/NOUN-MS ثاني/ADJ-MS E/E',
 'S/S في/PREP خطاب/NOUN-MS آخر/ADJ-MS

In [24]:
POSDict = utils.tagsDict(POSTaggedParagraphs, 'pos')

for w in POSDict:
    print(w)

{'S': 'S', 'صف': 'NOUN-MS', 'مرشح': 'ADJ-MS', 'جمهور': 'NOUN-MS', 'ل+': 'PREP', 'ال+رءاسه': 'DET+NOUN-MS', 'ال+اميركيه': 'DET+ADJ-MS', 'دونالد': 'NOUN-MS', 'ترامب': 'NOUN-MS', 'مهاجر': 'NOUN-MS', 'شرعي': 'ADJ-MS', 'ولاي+ة': 'NOUN+NSUFF-FS', 'المتحده': 'NOUN-MS', 'بان': 'NOUN-MP', 'حيوان': 'NOUN-MS', 'ليس+وا': 'V+PRON', 'بشر': 'NOUN-MS', 'خلال': 'NOUN-MS', 'خطاب': 'NOUN-MS', 'ألقا': 'V', '+ه': 'PRON', 'و+': 'CONJ', 'لا': 'PART', '+ي': 'PRON', 'ميشيغان': 'NOUN-MS', 'اثار': 'NOUN-FS', 'انتقاد': 'NOUN-MS', 'واسع': 'ADJ-MS', 'عود': 'NOUN-MS', 'ال': 'DET', 'ال+لغه': 'DET+ADJ-MS', 'ال+مهينه': 'DET+ADJ-MS', 'اعتمد': 'V', 'حمل': 'NOUN-MS', 'ال+سابقه': 'DET+NOUN-MS', 'E': 'E'}
{'S': 'S', 'جاء': 'V', 'تصريح': 'NOUN-MS', 'ترامب': 'NOUN-MS', 'خلال': 'NOUN-MS', 'ظهور': 'NOUN-MS', 'ال': 'DET', 'جانب': 'NOUN-MS', 'عدد': 'NOUN-MS', 'ضباط': 'NOUN-MP', 'إنفاذ': 'NOUN-MS', 'قانون': 'NOUN-MS', 'تطرق': 'V', 'عد': 'NOUN-MS', 'قضايا': 'NOUN-FP', 'جناءيه': 'NOUN-MS', 'تعلق': 'V', 'مشتبه': 'NOUN-MS', 'موجود': '

In [27]:
MyOwnTaggedParagraphs = []

for p in contentWordsLemmatized:
    MyOwnTaggedParagraphs.append(tagger.dumbPOSTagger(' '.join(p)))

for p in MyOwnTaggedParagraphs:
    print(p)

{'صف': 'NOUN', 'مرشح': 'NOUN', 'جمهور': 'NOUN', 'للرءاسه': 'NOUN', 'الاميركيه': 'NOUN', 'دونالد': 'NOUN', 'ترامب': 'NOUN', 'مهاجر': 'NOUN', 'شرعي': 'NOUN', 'ولاية': 'NOUN', 'المتحده': 'NOUN', 'بان': 'NOUN', 'حيوان': 'NOUN', 'ليسوا': 'VERB', 'بشر': 'NOUN', 'خلال': 'NOUN', 'خطاب': 'NOUN', 'القاه': 'NOUN', 'ولايه': 'NOUN', 'ميشيغان': 'NOUN', 'اثار': 'NOUN', 'انتقاد': 'NOUN', 'واسع': 'NOUN', 'عود': 'NOUN', 'ال': 'NOUN', 'اللغه': 'NOUN', 'المهينه': 'NOUN', 'اعتمد': 'VERB', 'حمل': 'NOUN', 'السابقه': 'NOUN'}
{'جاء': 'NOUN', 'تصريح': 'NOUN', 'ترامب': 'NOUN', 'خلال': 'NOUN', 'ظهور': 'ADJ', 'ال': 'NOUN', 'جانب': 'NOUN', 'عدد': 'NOUN', 'ضباط': 'NOUN', 'انفاذ': 'VERB', 'قانون': 'NOUN', 'تطرق': 'NOUN', 'عد': 'NOUN', 'قضايا': 'NOUN', 'جناءيه': 'NOUN', 'تعلق': 'NOUN', 'مشتبه': 'NOUN', 'موجود': 'NOUN', 'ولاية': 'NOUN', 'المتحده': 'NOUN', 'شكل': 'NOUN', 'حذر': 'NOUN', 'ان': 'NOUN', 'عنف': 'NOUN', 'والفوضي': 'NOUN', 'دمر': 'NOUN', 'بلاد': 'NOUN', 'اذا': 'NOUN', 'أحق': 'NOUN', 'فوز': 'NOUN', 'انتخاب': 'N

## **Named Entity Recognition (NER) with Farasa**

In [47]:
named_entity_recognizer = FarasaNamedEntityRecognizer()

NamedEntityRecognized = named_entity_recognizer.recognize(stemmedTextFarasa)
print("sample named entity recognized:\n", NamedEntityRecognized)

sample named entity recognized:
 وصف/O مرشح/O جمهوري/O رءاسه/O اميركيه/O دونالد/B-PERS ترامب/I-PERS مهاجر/O غير/O شرعي/O في/O ولاية/O المتحده/B-PERS أن/O حيوان/O ليس/O بشر/O خلال/O خطاب/O ألقى/O في/O لا/O ميشيغان/B-LOC من/O اثار/O انتقاد/O واسع/O عودة/O الي/O لغه/O مهينه/O الذي/O اعتمد/O في/O حملة/O سابق/O
جاء/O تصريح/O ترامب/O خلال/O ظهور/O الي/O جانب/O عدد/O من/O ضابط/O إنفاذ/O قانون/O حيث/O تطرق/O الي/O عد/O قضية/O جناءيه/O تعلق/O مشتبه/O ب/O موجود/O في/O ولاية/O المتحده/O شكل/O غير/O قانوني/O حذر/O من/O ان/O عنف/O فوضي/O دمر/O بلد/O اذ/O لم/O حقق/O فوز/O في/O انتخاب/O مقرره/O في/O نوفمبرتشرين/O ثاني/O
في/O خطاب/O آخر/O في/O لا/O ويسكونسن/O واصل/O ترامب/O نبر/B-PERS تصريح/O مثيره/O حيث/O وصف/O انتخاب/O عام/O 2024/O أن/O معركه/O نهاءيه/O لم/O
في/O تعلق/O حادث/O لاك/O رايلي/B-PERS طالب/O تمريض/O بالغ/O من/O عمر/O 22/O عام/O من/O جورجيا/B-LOC الذي/O زعم/O ان/O قتل/O علي/O يد/O مهاجر/O غير/O شرعي/O من/O فنزويلا/B-LOC أشار/O ترامب/O الي/O ان/O بعض/O مهاجر/O ليس/O بشر/O
في/O تعليق/O قال/O

In [48]:
NERParagraphs = NamedEntityRecognized.split('\n')

NERParagraphs = list(filter(None, NERParagraphs))
res = []
for ele in NERParagraphs:
    if ele.strip():
        res.append(ele)

NERParagraphs = res
NERParagraphs

['وصف/O مرشح/O جمهوري/O رءاسه/O اميركيه/O دونالد/B-PERS ترامب/I-PERS مهاجر/O غير/O شرعي/O في/O ولاية/O المتحده/B-PERS أن/O حيوان/O ليس/O بشر/O خلال/O خطاب/O ألقى/O في/O لا/O ميشيغان/B-LOC من/O اثار/O انتقاد/O واسع/O عودة/O الي/O لغه/O مهينه/O الذي/O اعتمد/O في/O حملة/O سابق/O',
 'جاء/O تصريح/O ترامب/O خلال/O ظهور/O الي/O جانب/O عدد/O من/O ضابط/O إنفاذ/O قانون/O حيث/O تطرق/O الي/O عد/O قضية/O جناءيه/O تعلق/O مشتبه/O ب/O موجود/O في/O ولاية/O المتحده/O شكل/O غير/O قانوني/O حذر/O من/O ان/O عنف/O فوضي/O دمر/O بلد/O اذ/O لم/O حقق/O فوز/O في/O انتخاب/O مقرره/O في/O نوفمبرتشرين/O ثاني/O',
 'في/O خطاب/O آخر/O في/O لا/O ويسكونسن/O واصل/O ترامب/O نبر/B-PERS تصريح/O مثيره/O حيث/O وصف/O انتخاب/O عام/O 2024/O أن/O معركه/O نهاءيه/O لم/O',
 'في/O تعلق/O حادث/O لاك/O رايلي/B-PERS طالب/O تمريض/O بالغ/O من/O عمر/O 22/O عام/O من/O جورجيا/B-LOC الذي/O زعم/O ان/O قتل/O علي/O يد/O مهاجر/O غير/O شرعي/O من/O فنزويلا/B-LOC أشار/O ترامب/O الي/O ان/O بعض/O مهاجر/O ليس/O بشر/O',
 'في/O تعليق/O قال/O ترامب/O قال/O 

In [49]:
NamedEntityRecognizedWords = utils.tagsDict(NERParagraphs, 'ner')

for w in NamedEntityRecognizedWords:
    print(w)

{'وصف': 'O', 'مرشح': 'O', 'جمهوري': 'O', 'رءاسه': 'O', 'اميركيه': 'O', 'دونالد': 'B-PERS', 'ترامب': 'I-PERS', 'مهاجر': 'O', 'غير': 'O', 'شرعي': 'O', 'في': 'O', 'ولاية': 'O', 'المتحده': 'B-PERS', 'أن': 'O', 'حيوان': 'O', 'ليس': 'O', 'بشر': 'O', 'خلال': 'O', 'خطاب': 'O', 'ألقى': 'O', 'لا': 'O', 'ميشيغان': 'B-LOC', 'من': 'O', 'اثار': 'O', 'انتقاد': 'O', 'واسع': 'O', 'عودة': 'O', 'الي': 'O', 'لغه': 'O', 'مهينه': 'O', 'الذي': 'O', 'اعتمد': 'O', 'حملة': 'O', 'سابق': 'O'}
{'جاء': 'O', 'تصريح': 'O', 'ترامب': 'O', 'خلال': 'O', 'ظهور': 'O', 'الي': 'O', 'جانب': 'O', 'عدد': 'O', 'من': 'O', 'ضابط': 'O', 'إنفاذ': 'O', 'قانون': 'O', 'حيث': 'O', 'تطرق': 'O', 'عد': 'O', 'قضية': 'O', 'جناءيه': 'O', 'تعلق': 'O', 'مشتبه': 'O', 'ب': 'O', 'موجود': 'O', 'في': 'O', 'ولاية': 'O', 'المتحده': 'O', 'شكل': 'O', 'غير': 'O', 'قانوني': 'O', 'حذر': 'O', 'ان': 'O', 'عنف': 'O', 'فوضي': 'O', 'دمر': 'O', 'بلد': 'O', 'اذ': 'O', 'لم': 'O', 'حقق': 'O', 'فوز': 'O', 'انتخاب': 'O', 'مقرره': 'O', 'نوفمبرتشرين': 'O', 'ثاني': 'O'}

In [50]:
NamedEntityRecognizedWords = utils.tagsDict(NERParagraphs, 'ner', True)

for w in NamedEntityRecognizedWords:
    print(w)

{'دونالد': 'B-PERS', 'ترامب': 'I-PERS', 'المتحده': 'B-PERS', 'ميشيغان': 'B-LOC'}
{}
{'نبر': 'B-PERS'}
{'رايلي': 'B-PERS', 'جورجيا': 'B-LOC', 'فنزويلا': 'B-LOC'}
{}
{'المتحده': 'B-LOC'}
{'جو': 'B-PERS', 'بايدن': 'I-PERS', 'كونجرس': 'B-ORG'}
{'مايكل': 'B-PERS', 'تايلر': 'I-PERS', 'بايدن': 'B-PERS', 'المتحده': 'B-LOC'}
{'رويترزابسوس': 'B-PERS', 'نهاي': 'I-PERS', 'فبرايرشباط': 'B-PERS', 'ماضي': 'I-PERS', 'المتحده': 'B-LOC'}
{}
{'ميشيغان': 'B-PERS', 'ويسكنسن': 'I-PERS', 'بايدن': 'B-PERS', 'ترامب': 'I-PERS'}
{'بايدن': 'B-PERS', 'ويسكنسن': 'B-LOC', 'ميشيغان': 'B-LOC'}


In [109]:
# Farasa WEB API - NOT WORKING

# conn = http.client.HTTPSConnection("farasa.qcri.org") 
# payload = "{'text': 'أظهر استطلاع أجرة رويترزابسوس نهاي فبرايرشباط ماضي', 'api_key': 'lonewolfdummy@gmail.com'}"
# headers = { "content-type": "application/json", "cache-control": "no-cache", }
# conn.request("POST", "/webapi/lemmatization/", payload.encode('utf-8'), headers)
# res = conn.getresponse()
# data = res.read()
# print(data.decode("utf-8"))

# url = 'https://farasa.qcri.org/webapi/lemmatization/'
# text = 'يُشار إلى أن اللغة العربية' 
# api_key = "#####################"
# payload = {'text': text, 'api_key': api_key}
# data = requests.post(url, data=payload)
# result = json.loads(data.text)
# print(result) 

# **REFERENCES**
[1] Motazsaad, “GitHub - motazsaad/process-arabic-text: Pre-process arabic text (remove diacritics, punctuations and repeating characters),” GitHub. https://github.com/motazsaad/process-arabic-text/

[2] T. Zerrouki, “pyarabic, an arabic language library for python.”, 2010. [Online]. Available: https://pypi.python.org/pypi/pyarabic/

[3] ——, “Tashaphyne, arabic light stemmer,” 2012. [Online]. Available: https://pypi.python.org/pypi/Tashaphyne/

[4] ——, “qalsadi, arabic mophological analyzer library for python.” 2012. [Online]. Available: https://pypi.python.org/pypi/qalsadi/

[5] ——, “Towards an open platform for arabic language processing,” 2020.

[6] A. L. T. G. at Qatar Computing Research Institute (QCRI), “farasa, the state-of-the-art full-stack package to deal with arabic language processing.” 2020. [Online]. Available: https://farasa.qcri.org/

[7] MagedSaeed, “GitHub - MagedSaeed/farasapy: A Python implementation of Farasa toolkit,” GitHub. https://github.com/MagedSaeed/farasapy/

[8] M. H. Btoush, A. Alarabeyyat, and I. Olab, “Rule based approach for arabic part of speech tagging and name entity recognition,” International Journal of Advanced Computer Science and Applications, vol. 7, no. 6, 2016. [Online]. Available: http://dx.doi.org/10.14569/IJACSA.2016.070642/